In [1]:
import pandas as pd

data = pd.read_csv("../../data/twitter15-tf_dataset.csv", lineterminator="\n")
print(data.shape)
data.head()

(742, 4)


,tweet_id,tweet_text,label,tvt2
0,551099691702956032,rip elly may clampett: so sad to learn #beverl...,True,training
1,514106273852174337,just in: missing afghan soldiers found trying ...,True,training
2,495366618818830336,#riphulkhogan my heart is ripping like your sh...,False,training
3,532206910796468224,a chick-fil-a manager allegedly banned this hi...,False,training
4,560187970389819392,islamic tribunal using sharia law in texas has...,False,validation


In [2]:
length = data.shape[0]
length

742

In [3]:
import random

# random.seed(33)

train_w = [0 for i in range(675)]
val_w = [1 for i in range(225)]
test_w = [2 for i in range(100)]

weights = train_w + val_w + test_w
print(f"weights : {len(weights)}")

tvt = []
for i in range(length):
    gacha = random.sample(weights, 1)[0]
    if gacha == 0:
        tvt.append("training")
    elif gacha == 1:
        tvt.append("validation")
    else:
        tvt.append("testting")

print(f"Training : {tvt.count('training')} - {round(tvt.count('training')/length, 3)}")
print(f"Validation : {tvt.count('validation')} - {round(tvt.count('validation')/length, 3)}")
print(f"Testing : {tvt.count('testting')} - {round(tvt.count('testting')/length, 3)}")

weights : 1000
Training : 497 - 0.67
Validation : 152 - 0.205
Testing : 93 - 0.125


In [4]:
data['tvt2'] = pd.Series(tvt)
data.head()

,tweet_id,tweet_text,label,tvt2
0,551099691702956032,rip elly may clampett: so sad to learn #beverl...,True,training
1,514106273852174337,just in: missing afghan soldiers found trying ...,True,training
2,495366618818830336,#riphulkhogan my heart is ripping like your sh...,False,training
3,532206910796468224,a chick-fil-a manager allegedly banned this hi...,False,validation
4,560187970389819392,islamic tribunal using sharia law in texas has...,False,validation


In [5]:
def bool2str(df):
    if df['label'] == True:
        return "true"
    else:
        return "false"

data['label'] = data.apply(lambda row: bool2str(row), axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742 entries, 0 to 741
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweet_id    742 non-null    int64 
 1   tweet_text  742 non-null    object
 2   label       742 non-null    object
 3   tvt2        742 non-null    object
dtypes: int64(1), object(3)
memory usage: 23.3+ KB


In [6]:
label_count = data['label'].value_counts()
label_count

true     372
false    370
Name: label, dtype: int64

In [7]:
combination = data.apply(lambda row: f"{row['label']}_{row['tvt2']}", axis=1).value_counts()
comparison = {}
for k, comb in combination.items():
    cv_fold = k.split("_")[1]
    label = k.split("_")[0]

    if cv_fold not in comparison:
        comparison[cv_fold] = {}
    
    comparison[cv_fold][label] = comb

labels = data['label'].unique().tolist()

def label_ratio(label_dict, labels):
    total = sum([v for k, v in label_dict.items()])
    
    report = ""
    for l in labels:
        report += f"{round(label_dict[l]/total, 2)}\t"
    
    return report[:-1]

def label_raw_value(label_dict, labels):
    total = sum([v for k, v in label_dict.items()])
    
    report = ""
    for l in labels:
        report += f"{label_dict[l]}\t"
    
    return report[:-1]

labels_str = ','.join([str(l) for l in labels])
print(f"\nLabel,{labels_str}")
print(f"Original\t{label_ratio(label_count, labels)}")
for cv, comp in comparison.items():
    print(f"{cv.title()}\t{label_ratio(comp, labels)}")
    
labels_str = ','.join([str(l) for l in labels])
print(f"\nLabel,{labels_str}")
print(f"Original\t{label_raw_value(label_count, labels)}")
for cv, comp in comparison.items():
    print(f"{cv.title()}\t{label_raw_value(comp, labels)}")


Label,true,false
Original	0.5	0.5
Training	0.49	0.51
Validation	0.53	0.47
Testting	0.49	0.51

Label,true,false
Original	372	370
Training	246	251
Validation	80	72
Testting	46	47


In [9]:
data.to_csv("../../data/twitter15-tf_dataset.csv", index=False)